In [62]:
import albumentations as A
import cv2
import json
import numpy as np

In [63]:
transform = A.Compose([
    A.RandomCrop(width=450, height=450),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
], bbox_params=A.BboxParams(format='coco'))

In [64]:
img = cv2.imread("../data/raw/nano/Rot/Rot1.jpg")


In [65]:
json_file_path = "../data/raw/nano/Rot/Lego_Parts_Labeling.json"

with open(json_file_path, "r") as file:
    json_data = json.load(file)
    
region_data = json_data["_via_img_metadata"]['Rot1.jpg27430']['regions'][0]['shape_attributes']
x = region_data['x']
y = region_data['y']
width = region_data['width']
height = region_data['height']

# Ausgabe der Werte
print("x:", x)
print("y:", y)
print("width:", width)
print("height:", height)


x: 274
y: 248
width: 76
height: 68


In [66]:
# coco [x_min, y_min, width, height]
bboxes =[
    [x, y, width, height, 'Rot']
    ]


In [67]:
transformed = transform(image=img, bboxes=bboxes)
transformed_image = transformed['image']
transformed_bboxes = transformed['bboxes']

# Kopie des Bildes erstellen, um die Bounding Box zu zeichnen
image_with_bboxes = transformed_image.copy()

# Bounding Box zeichnen
bbox = transformed_bboxes[0]  # Annahme: Es gibt nur eine Bounding Box
x, y, width, height, label = bbox
cv2.rectangle(image_with_bboxes, (int(x), int(y)), (int(x + width), int(y + height)), (0, 255, 0), 2)
cv2.putText(image_with_bboxes, label, (int(x), int(y) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

# Bild mit gezeichneter Bounding Box anzeigen
cv2.imshow("Bild mit Bounding Box", image_with_bboxes)
cv2.waitKey(0)
cv2.destroyAllWindows()